# Analyze

In [1]:
import pandas as pd
import os
import json

In [2]:
datasets = [
    "regress_plane",
    "regress_gaussian",
    "classify_two_gauss",
    "classify_spiral",
    "classify_circle",
    "classify_xor"
]

models = [
    "MLP_Relu_1h",
    "MLP_Relu_2h",
    "MLP_Silu_1h",
    "MLP_Silu_2h",
    "MLP_Tanh_1h",
    "MLP_Tanh_2h",
    "Feat_Attn_1h",
    "Feat_Attn_2h",
    "Feat_Attn_3h"
]

optimizers = [
    "Adam",
    "SGD"
]

splits = [
    "train",
    "val",
    "test"
]

In [3]:
results = []
metrics_file = "metrics.json"
# gen a min loss markdown table here
for split in splits:
    for dataset in datasets:
        for optimizer in optimizers:
            for model in models:
                result_dir = f"../result/{dataset}_{model}_{optimizer}"
                with open(os.path.join(result_dir, metrics_file), 'r') as file:
                    metrics = json.load(file)
                metrics_df = pd.DataFrame(metrics)

                min_loss = min(metrics_df[split])
                results.append([split, dataset, model, optimizer, min_loss])


# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Split", "Dataset", "Model", "Optimizer", "Min Loss"])

# Convert DataFrame to Markdown table
markdown_table = results_df.to_markdown(index=False)

# Save the DataFrame to a CSV file
results_df.to_csv("min_loss.csv", index=False)

print("Analysis report generated: min_loss.csv")

Analysis report generated: min_loss.csv


In [4]:
results_df.head()

,Split,Dataset,Model,Optimizer,Min Loss
0,train,regress_plane,MLP_Relu_1h,Adam,0.006407
1,train,regress_plane,MLP_Relu_2h,Adam,0.006309
2,train,regress_plane,MLP_Silu_1h,Adam,0.006219
3,train,regress_plane,MLP_Silu_2h,Adam,0.006013
4,train,regress_plane,MLP_Tanh_1h,Adam,0.006607


In [5]:
results_df.keys()

Index(['Split', 'Dataset', 'Model', 'Optimizer', 'Min Loss'], dtype='object')

In [6]:
for key in results_df.keys()[:-1]:
    print(key, results_df[key].unique())

Split ['train' 'val' 'test']
Dataset ['regress_plane' 'regress_gaussian' 'classify_two_gauss' 'classify_spiral'
 'classify_circle' 'classify_xor']
Model ['MLP_Relu_1h' 'MLP_Relu_2h' 'MLP_Silu_1h' 'MLP_Silu_2h' 'MLP_Tanh_1h'
 'MLP_Tanh_2h' 'Feat_Attn_1h' 'Feat_Attn_2h' 'Feat_Attn_3h']
Optimizer ['Adam' 'SGD']


In [7]:
benchmarks = [f"{dataset}&{split}&{optimizer}" for dataset in datasets for split in splits for optimizer in optimizers]
rank_df = pd.DataFrame(index=models, columns=benchmarks)

# sort for each benchmark
for benchmark in benchmarks:
    data, split, optim = benchmark.split("&")
    benchmark_df = results_df[(results_df["Split"] == split) & (results_df["Dataset"] == data) & (results_df["Optimizer"] == optim)]
    benchmark_df = benchmark_df.sort_values(by="Min Loss", ascending=True)
    for i, row in benchmark_df.iterrows():
        rank_df.at[row["Model"], benchmark] = benchmark_df.index.get_loc(i) + 1

rank_df


,regress_plane&train&Adam,regress_plane&train&SGD,regress_plane&val&Adam,regress_plane&val&SGD,regress_plane&test&Adam,regress_plane&test&SGD,regress_gaussian&train&Adam,regress_gaussian&train&SGD,regress_gaussian&val&Adam,regress_gaussian&val&SGD,...,classify_circle&val&Adam,classify_circle&val&SGD,classify_circle&test&Adam,classify_circle&test&SGD,classify_xor&train&Adam,classify_xor&train&SGD,classify_xor&val&Adam,classify_xor&val&SGD,classify_xor&test&Adam,classify_xor&test&SGD
MLP_Relu_1h,5,6,5,5,8,3,5,8,7,8,...,3,8,6,9,8,1,6,9,1,6
MLP_Relu_2h,4,2,1,8,6,4,7,7,8,9,...,2,6,9,2,5,2,8,2,4,1
MLP_Silu_1h,3,1,4,2,3,7,8,9,9,7,...,6,4,1,4,7,3,1,1,3,7
MLP_Silu_2h,2,3,6,3,9,2,2,1,1,4,...,8,7,7,5,9,7,3,7,8,8
MLP_Tanh_1h,7,5,8,6,4,5,3,2,2,6,...,9,9,5,8,1,6,9,8,9,9
MLP_Tanh_2h,1,9,3,4,7,8,4,3,6,1,...,5,2,3,1,6,4,7,3,2,2
Feat_Attn_1h,8,8,7,7,5,1,6,4,4,3,...,1,5,8,6,4,8,4,4,6,5
Feat_Attn_2h,6,4,9,1,2,6,1,5,5,2,...,7,1,4,3,3,9,5,6,5,4
Feat_Attn_3h,9,7,2,9,1,9,9,6,3,5,...,4,3,2,7,2,5,2,5,7,3


In [8]:
train_benchmarks = [f"{dataset}&train&{optimizer}" for dataset in datasets for optimizer in optimizers]
val_benchmarks = [f"{dataset}&val&{optimizer}" for dataset in datasets for optimizer in optimizers]
test_benchmarks = [f"{dataset}&test&{optimizer}" for dataset in datasets for optimizer in optimizers]
train_rank_df = rank_df[train_benchmarks]
val_rank_df = rank_df[val_benchmarks]
test_rank_df = rank_df[test_benchmarks]

In [9]:
train_rank_df

,regress_plane&train&Adam,regress_plane&train&SGD,regress_gaussian&train&Adam,regress_gaussian&train&SGD,classify_two_gauss&train&Adam,classify_two_gauss&train&SGD,classify_spiral&train&Adam,classify_spiral&train&SGD,classify_circle&train&Adam,classify_circle&train&SGD,classify_xor&train&Adam,classify_xor&train&SGD
MLP_Relu_1h,5,6,5,8,5,1,6,6,4,2,8,1
MLP_Relu_2h,4,2,7,7,6,2,2,3,5,6,5,2
MLP_Silu_1h,3,1,8,9,3,6,9,9,1,7,7,3
MLP_Silu_2h,2,3,2,1,2,8,1,5,2,5,9,7
MLP_Tanh_1h,7,5,3,2,1,5,7,7,8,8,1,6
MLP_Tanh_2h,1,9,4,3,7,4,5,8,3,3,6,4
Feat_Attn_1h,8,8,6,4,9,3,4,2,7,4,4,8
Feat_Attn_2h,6,4,1,5,8,7,3,1,9,1,3,9
Feat_Attn_3h,9,7,9,6,4,9,8,4,6,9,2,5


In [10]:
val_rank_df

,regress_plane&val&Adam,regress_plane&val&SGD,regress_gaussian&val&Adam,regress_gaussian&val&SGD,classify_two_gauss&val&Adam,classify_two_gauss&val&SGD,classify_spiral&val&Adam,classify_spiral&val&SGD,classify_circle&val&Adam,classify_circle&val&SGD,classify_xor&val&Adam,classify_xor&val&SGD
MLP_Relu_1h,5,5,7,8,4,6,5,5,3,8,6,9
MLP_Relu_2h,1,8,8,9,9,8,3,3,2,6,8,2
MLP_Silu_1h,4,2,9,7,6,5,7,4,6,4,1,1
MLP_Silu_2h,6,3,1,4,7,4,1,6,8,7,3,7
MLP_Tanh_1h,8,6,2,6,2,9,9,9,9,9,9,8
MLP_Tanh_2h,3,4,6,1,5,1,6,7,5,2,7,3
Feat_Attn_1h,7,7,4,3,8,3,4,2,1,5,4,4
Feat_Attn_2h,9,1,5,2,3,7,2,1,7,1,5,6
Feat_Attn_3h,2,9,3,5,1,2,8,8,4,3,2,5


In [11]:
test_rank_df

,regress_plane&test&Adam,regress_plane&test&SGD,regress_gaussian&test&Adam,regress_gaussian&test&SGD,classify_two_gauss&test&Adam,classify_two_gauss&test&SGD,classify_spiral&test&Adam,classify_spiral&test&SGD,classify_circle&test&Adam,classify_circle&test&SGD,classify_xor&test&Adam,classify_xor&test&SGD
MLP_Relu_1h,8,3,3,8,4,3,7,8,6,9,1,6
MLP_Relu_2h,6,4,9,3,2,4,2,3,9,2,4,1
MLP_Silu_1h,3,7,7,9,1,8,8,6,1,4,3,7
MLP_Silu_2h,9,2,1,5,9,7,1,9,7,5,8,8
MLP_Tanh_1h,4,5,6,1,5,9,9,7,5,8,9,9
MLP_Tanh_2h,7,8,2,2,8,1,5,5,3,1,2,2
Feat_Attn_1h,5,1,5,6,3,2,4,2,8,6,6,5
Feat_Attn_2h,2,6,4,4,6,6,3,1,4,3,5,4
Feat_Attn_3h,1,9,8,7,7,5,6,4,2,7,7,3


In [12]:
rank_split_df = pd.DataFrame(index=models, columns=splits)
rank_split_df["train"] = train_rank_df.mean(axis=1).rank()
rank_split_df["val"] = val_rank_df.mean(axis=1).rank()
rank_split_df["test"] = test_rank_df.mean(axis=1).rank()
rank_split_df

,train,val,test
MLP_Relu_1h,4.0,8.0,6.5
MLP_Relu_2h,2.0,7.0,3.0
MLP_Silu_1h,7.0,5.0,5.0
MLP_Silu_2h,1.0,6.0,8.0
MLP_Tanh_1h,6.0,9.0,9.0
MLP_Tanh_2h,4.0,2.0,1.0
Feat_Attn_1h,8.0,3.5,4.0
Feat_Attn_2h,4.0,1.0,2.0
Feat_Attn_3h,9.0,3.5,6.5


In [13]:
# add a row as the final rank, 
# since the rank of train focuses on the degree of fitting and the ability of the model to learn easily, 
# while the rank of val and test focuses on the generalization ability of the model
rank_split_df["score"] = rank_split_df['train'] * 0.4 + rank_split_df['val'] * 0.3 + rank_split_df['test'] * 0.3
rank_split_df.sort_values(by="score", ascending=True)

,train,val,test,score
MLP_Tanh_2h,4.0,2.0,1.0,2.50
Feat_Attn_2h,4.0,1.0,2.0,2.50
MLP_Relu_2h,2.0,7.0,3.0,3.80
MLP_Silu_2h,1.0,6.0,8.0,4.60
Feat_Attn_1h,8.0,3.5,4.0,5.45
MLP_Silu_1h,7.0,5.0,5.0,5.80
MLP_Relu_1h,4.0,8.0,6.5,5.95
Feat_Attn_3h,9.0,3.5,6.5,6.60
MLP_Tanh_1h,6.0,9.0,9.0,7.80


In [14]:
with open("rank.md", "w") as file:
    final = rank_split_df.sort_values(by="score", ascending=True)
    final["rank"] = final["score"].rank()
    file.write(final.to_markdown())
    print("Ranking report generated: rank.md")

Ranking report generated: rank.md
